In [30]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

np.seterr(all='ignore')

OPERATORS = ['+', '-', '*', '/']
FUNCTIONS = ['np.sin', 'np.cos', 'np.exp', 'np.log', 'np.sqrt', 'np.abs']

class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __str__(self):
        if not self.left and not self.right:
            return str(self.value)
        if self.right and not self.left:
            return f"{self.value}({self.right})"  # Nodo con funzione unaria
        left_str = str(self.left)
        right_str = str(self.right)
        return f"({left_str} {self.value} {right_str})"  # Nodo con operatore binario
    
    def is_operator(self):
        return self.value in OPERATORS

    def is_function(self):
        return self.value in FUNCTIONS

    def is_leaf(self):
        return not self.is_operator() and not self.is_function()
    
    def copy(self):
        # Creiamo un nuovo nodo e copiamo i riferimenti ricorsivamente
        return Node(
            self.value, 
            self.left.copy() if self.left else None, 
            self.right.copy() if self.right else None
        )
    
    def depth(self):
        # Se il nodo è foglia (nessun figlio), la profondità è 0
        if self.left is None and self.right is None:
            return 0
        # Altrimenti la profondità è 1 + la profondità massima tra i figli
        left_depth = self.left.depth() if self.left else 0
        right_depth = self.right.depth() if self.right else 0
        return 1 + max(left_depth, right_depth)

    def evaluate(self, variable_values):
        if self.value in OPERATORS:
            left_val = self.left.evaluate(variable_values) if self.left else 0
            right_val = self.right.evaluate(variable_values) if self.right else 0
            
            if self.value == "+":
                return left_val + right_val
            elif self.value == "-":
                return left_val - right_val
            elif self.value == "*":
                return left_val * right_val
            elif self.value == "/":
                return left_val / right_val if right_val != 0 else float('inf')
        
        elif self.value in FUNCTIONS:
            arg = self.right.evaluate(variable_values)
            if self.value == "np.sin":
                return np.sin(arg)
            elif self.value == "np.cos":
                return np.cos(arg)
            elif self.value == "np.exp":
                return np.exp(arg)
            elif self.value == "np.log":
                return np.log(arg) if arg > 0 else float('inf')
            elif self.value == "np.sqrt":
                return np.sqrt(arg) if arg >= 0 else float('inf')
            elif self.value == "np.abs":
                return np.abs(arg)
        
        elif self.value.startswith('x'):
            idx = int(self.value[2:-1])
            return variable_values[f"x[{idx}]"]
        
        else:
            try:
                return float(self.value)
            except ValueError:
                raise ValueError(f"Il valore {self.value} non è un valore valido")


In [31]:
def generate_population(X, pop_size):
    num_variables = X.shape[0]
    population = []
    for _ in range(pop_size):
        depth = random.randint(num_variables, 10)
        formula = generate_formula(num_variables, depth)
        population.append(formula)
    return population


def generate_formula(num_variables, depth=3):
    if depth == 0:
        # Foglia: una variabile o una costante
        variables = [f"x[{i}]" for i in range(num_variables)]
        return Node(random.choice(variables + [str(round(random.uniform(-10, 10), 2))]))

    if random.random() < 0.5:
        # Nodo funzione unaria
        function = random.choice(FUNCTIONS)
        child = generate_formula(num_variables, depth - 1)
        return Node(function, right=child)
    else:
        # Nodo operatore binario
        operator = random.choice(OPERATORS)
        left_child = generate_formula(num_variables, depth - 1)
        right_child = generate_formula(num_variables, depth - 1)
        return Node(operator, left_child, right_child)


def evaluate_fitness(formula, X, Y):
    predictions = []
    
    for idx in range(X.shape[1]):
        variables = {f"x[{i}]": X[i, idx] for i in range(X.shape[0])}
        prediction = formula.evaluate(variables)
        predictions.append(prediction)
    
    predictions = np.array(predictions)

    # controllo se ci sono valori NaN
    if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
        return float('inf')
    try:
        mse = np.mean((predictions - Y) ** 2)
        return mse
    except Exception:  # Cattura gli errori di overflow
        print("Overflow error encountered. Returning a high MSE.")
        return float('inf')


def get_all_nodes(node):
    if node is None:
        return []
    nodes = [node]
    nodes += get_all_nodes(node.left)  # Aggiungi nodi del sotto-albero sinistro
    nodes += get_all_nodes(node.right)  # Aggiungi nodi del sotto-albero destro
    return nodes


def mutate(formula, num_variables, mutation_rate=0.2):
    if random.random() < mutation_rate:
        selected_node = random.choice(get_all_nodes(formula))
        new_node = generate_formula(num_variables, selected_node.depth())
        selected_node.value, selected_node.left, selected_node.right = new_node.value, new_node.left, new_node.right

        
    else:
        if formula.left:
            mutate(formula.left, num_variables)
        if formula.right:
            mutate(formula.right, num_variables)
    return formula


def crossover(parent1, parent2):
    node1 = random.choice(get_all_nodes(parent1))
    node2 = random.choice(get_all_nodes(parent2))
    node1.value, node1.left, node1.right = node2.value, node2.left, node2.right
    return parent1, parent2


def limit_depth(formula, num_variables, max_depth=10):
    def trim(node, depth):
        if depth > max_depth:
            variables = [f"x[{i}]" for i in range(num_variables)]
            return Node(random.choice(variables + [str(round(random.uniform(-10, 10), 2))]))
        else:
            if node.left:
                node.left = trim(node.left, depth + 1)
            if node.right:
                node.right = trim(node.right, depth + 1)
            return node
    return trim(formula, 0)


def add_missing_variables(formula, num_variables):
    nodes = get_all_nodes(formula)
    leaf_nodes = [n for n in nodes if n.is_leaf()]
    leaf_values = [n.value for n in leaf_nodes]

    if len(leaf_values) >= num_variables:
        variables = [f"x[{i}]" for i in range(num_variables)]
        variables_in_formula = set(leaf_values)

        # Trova candidati alla sostituzione: costanti o variabili ripetute
        replace_candidates = []
        seen = set()
        for n in leaf_nodes:
            if not n.value.startswith("x") or n.value in seen:
                replace_candidates.append(n)
            else:
                seen.add(n.value)

        for variable in variables:
            if variable not in variables_in_formula and replace_candidates:
                node_to_replace = random.choice(replace_candidates)
                node_to_replace.value = variable

                # Aggiorna le liste
                variables_in_formula.add(variable)
                replace_candidates.remove(node_to_replace)
    
    return formula


def tournament_selection(population, fitness_scores, tournament_size=10):
    candidates = random.sample(list(zip(population, fitness_scores)), tournament_size)
    # Ordina per fitness (fitness più bassa è migliore se minimizziamo)
    candidates.sort(key=lambda x: x[1])
    return candidates[0][0].copy()

def gp(X, Y, pop_size, n_generations):
    # 1. Inizializza la popolazione casuale
    print('Generating population...')
    population = generate_population(X, pop_size)

    num_variables = X.shape[0]
    best_fitness_per_generation = []
    
    for g in tqdm(range(n_generations), desc='Generations'):
        # 2. Valuta la fitness di ogni formula
        fitness_scores = [evaluate_fitness(formula, X, Y) for formula in population]

        # Salva la fitness migliore di questa generazione
        best_idx = np.argmin(fitness_scores)
        best_formula = population[best_idx]
        best_fitness = fitness_scores[best_idx]
        print(f'Generation {g+1}: Best fitness = {best_fitness}, Formula = {best_formula}')
        best_fitness_per_generation.append(best_fitness)

        # 3. Elitismo: salva le migliori formule
        elitism_size = int(pop_size * 0.2)
        elitism = sorted(zip(population, fitness_scores), key=lambda x: x[1])
        elitism = [e.copy() for e, _ in elitism[:elitism_size]]

        next_population = elitism[:]
        
        # 4. Genera la nuova popolazione 
        while len(next_population) < pop_size:
            # Applica crossover
            parent1 = tournament_selection(population, fitness_scores, pop_size)
            parent2 = tournament_selection(population, fitness_scores, pop_size)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, num_variables)
            child2 = mutate(child2, num_variables)
            child1 = limit_depth(child1, num_variables)
            child2 = limit_depth(child2, num_variables)
            child1 = add_missing_variables(child1, num_variables)
            child2 = add_missing_variables(child2, num_variables)
            next_population.append(child1)
            if len(next_population) < pop_size:
                next_population.append(child2)

        if g % 10 == 0:
            population.extend(generate_population(X, int(pop_size*0.2)))

        # Sostituisci la popolazione
        population = next_population

    # Grafico dell'andamento della fitness
    plt.plot(best_fitness_per_generation)
    plt.title('Andamento della Fitness durante le Generazioni')
    plt.xlabel('Generazione')
    plt.ylabel('Fitness')
    plt.show()
    
    # 5. Trova la formula con la fitness migliore e ritorna la formula migliore e la fitness
    fitness_scores = [evaluate_fitness(formula, X, Y) for formula in population]
    best_idx = np.argmin(fitness_scores)
    best_formula = population[best_idx]
    best_fitness = fitness_scores[best_idx]
    return best_formula, best_fitness

In [32]:
def solution(problem_n, pop_size = 300, n_generations=100):
    print(f"Problem {problem_n}")
    problem = np.load(f'../data/problem_{problem_n}.npz')
    X = problem['x']
    Y = problem['y']
    best_formula, best_fitness = gp(X, Y, pop_size, n_generations)
    print("Best formula:", best_formula)
    print("Best fitness:", best_fitness)